#### 1. Загрузите данные из файла advertising.csv в объект pandas DataFrame.

In [1]:
import pandas as pd
adver_data = pd.read_csv('advertising.csv')

#### Посмотрите на первые 5 записей и на статистику признаков в этом наборе данных.


In [3]:
adver_data.head()

,TV,Radio,Newspaper,Sales
1,230.1,37.8,69.2,22.1
2,44.5,39.3,45.1,10.4
3,17.2,45.9,69.3,9.3
4,151.5,41.3,58.5,18.5
5,180.8,10.8,58.4,12.9


In [13]:
adver_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 1 to 200
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   Radio      200 non-null    float64
 2   Newspaper  200 non-null    float64
 3   Sales      200 non-null    float64
dtypes: float64(4)
memory usage: 7.8 KB


##### Создайте массивы NumPy X из столбцов TV, Radio и Newspaper и y - из столбца Sales. Используйте атрибут values объекта pandas DataFrame.


In [72]:
X = adver_data[['TV','Radio','Newspaper']].values
y = adver_data['Sales'].values

##### Отмасштабируйте столбцы матрицы X, вычтя из каждого значения среднее по соответствующему столбцу и поделив результат на стандартное отклонение. Для определенности, используйте методы mean и std векторов NumPy (реализация std в Pandas может отличаться). Обратите внимание, что в numpy вызов функции .mean() без параметров возвращает среднее по всем элементам массива, а не по столбцам, как в pandas. Чтобы произвести вычисление по столбцам, необходимо указать параметр axis.

In [73]:
means, stds = X.mean(axis =0) , X.std(axis = 0)

In [74]:
stds

array([85.63933176, 14.80964564, 21.72410606])

In [75]:
X = (X - means) / stds

#### Добавьте к матрице X столбец из единиц, используя методы hstack, ones и reshape библиотеки NumPy. Вектор из единиц нужен для того, чтобы не обрабатывать отдельно коэффициент  w0  линейной регрессии.


In [76]:
import numpy as np

X = np.hstack( (X , np.reshape(np.ones(200), (200,1))) )

#### 2.Реализуйте функцию mserror - среднеквадратичную ошибку прогноза. Она принимает два аргумента - объекты Series y (значения целевого признака) и y_pred (предсказанные значения). Не используйте в этой функции циклы - тогда она будет вычислительно неэффективной.

In [88]:
def mserror(y, y_pred):
    return ((y-y_pred)**2).mean()
    

#### Какова среднеквадратичная ошибка прогноза значений Sales, если всегда предсказывать медианное значение Sales по исходной выборке? Полученный результат, округленный до 3 знаков после запятой, является ответом на '1 задание'.


In [89]:
answer1 = mserror(y,y.mean())

print(round(answer1, 3))

27.086


#### 3. Реализуйте функцию normal_equation, которая по заданным матрицам (массивам NumPy) X и y вычисляет вектор весов  w  согласно нормальному уравнению линейной регрессии.

In [90]:
def normal_equation(X, y):
    return np.linalg.solve(np.dot(X.T,X),np.dot(X.T,y))

In [91]:
norm_eq_weights = normal_equation(X, y)
print(norm_eq_weights)

[ 3.91925365  2.79206274 -0.02253861 14.0225    ]


#### Какие продажи предсказываются линейной моделью с весами, найденными с помощью нормального уравнения, в случае средних инвестиций в рекламу по ТВ, радио и в газетах? (то есть при нулевых значениях масштабированных признаков TV, Radio и Newspaper). Полученный результат, округленный до 3 знаков после запятой, является ответом на '2 задание'.


In [103]:
X0 = np.hstack((np.zeros((len(X), 3)),np.ones((len(X),1))))
answer = X0.dot(norm_eq_weights)
answer2 = answer[1]
print(round(answer2, 3))

14.023


#### 4. Напишите функцию linear_prediction, которая принимает на вход матрицу X и вектор весов линейной модели w, а возвращает вектор прогнозов в виде линейной комбинации столбцов матрицы X с весами w.


In [104]:
def linear_prediction(X, w):
    return X.dot(w)

#### Какова среднеквадратичная ошибка прогноза значений Sales в виде линейной модели с весами, найденными с помощью нормального уравнения? Полученный результат, округленный до 3 знаков после запятой, является ответом на '3 задание'


In [105]:
w = normal_equation (X, y)
y_pred = linear_prediction(X, w)
answer3 = mserror(y, y_pred)
print(round(answer3, 3))

2.784


#### 5. Напишите функцию stochastic_gradient_step, реализующую шаг стохастического градиентного спуска для линейной регрессии. Функция должна принимать матрицу X, вектора y и w, число train_ind - индекс объекта обучающей выборки (строки матрицы X), по которому считается изменение весов, а также число  η  (eta) - шаг градиентного спуска (по умолчанию eta=0.01). Результатом будет вектор обновленных весов. Наша реализация функции будет явно написана для данных с 3 признаками, но несложно модифицировать для любого числа признаков, можете это сделать.
